In [17]:
import pandas as pd
from sklearn.impute import KNNImputer
import plotly.express as px
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim
import numpy as np
   

In [18]:
df = pd.read_csv("../statics/HDR21-22_Composite_indices_complete_time_series.csv")

In [19]:
df_numerics = df.select_dtypes(exclude = 'object')

In [20]:
knn_imp = KNNImputer(n_neighbors=20)
df_imputed = pd.DataFrame(knn_imp.fit_transform(df_numerics), columns=df_numerics.columns)

In [21]:
df[df_numerics.columns] = df_imputed

In [22]:
df_years = df[["country","hdi_1990","hdi_1991","hdi_1992","hdi_1993","hdi_1994","hdi_1995","hdi_1996","hdi_1997","hdi_1998","hdi_1999","hdi_2000","hdi_2001","hdi_2002","hdi_2003","hdi_2004","hdi_2005","hdi_2006",
                "hdi_2007","hdi_2008","hdi_2009","hdi_2010","hdi_2011","hdi_2012","hdi_2013","hdi_2014","hdi_2015","hdi_2016","hdi_2017","hdi_2018","hdi_2019","hdi_2020","hdi_2021"]]
df_years = df_years[:195]

In [23]:
df_years_mf = df[["country","mf_1990","mf_1991","mf_1992","mf_1993","mf_1994","mf_1995","mf_1996","mf_1997","mf_1998","mf_1999","mf_2000","mf_2001","mf_2002","mf_2003","mf_2004","mf_2005","mf_2006",
                "mf_2007","mf_2008","mf_2009","mf_2010","mf_2011","mf_2012","mf_2013","mf_2014","mf_2015","mf_2016","mf_2017","mf_2018","mf_2019","mf_2020","mf_2021"]]
df_years_mf = df_years_mf[:195]

In [24]:
years = [0]
for col in df_years.columns:
    time = col[-4:len(col)]
    try:
        time = int(time)
        if time not in years:
            years.append(time)
    except:
        continue

In [25]:
df_trans = df_years.transpose()
df_trans.reset_index(inplace=True)
df_trans["index"] = years

In [26]:
df_trans_mf = df_years_mf.transpose()
df_trans_mf.reset_index(inplace=True)
df_trans_mf["index"] = years

In [27]:
df_trans.rename(columns={"index":"years"}, inplace=True)
x = df_trans[df_trans.years == 0].drop("years", axis=1)
cols = []
for col in x.values:
    for i in col:
        cols.append(i)
x.columns = cols
y = df_trans.years
df_trans.drop("years", axis=1, inplace=True)
df_trans.columns = cols
df_trans["years"] = y
df_trans.drop(0, axis=0, inplace=True)
df_trans.reset_index(drop="index", inplace=True)

In [28]:
df_trans_mf.rename(columns={"index":"years"}, inplace=True)
x = df_trans_mf[df_trans_mf.years == 0].drop("years", axis=1)
cols = []
for col in x.values:
    for i in col:
        cols.append(i)
x.columns = cols
y = df_trans_mf.years
df_trans_mf.drop("years", axis=1, inplace=True)
df_trans_mf.columns = cols
df_trans_mf["years"] = y
df_trans_mf.drop(0, axis=0, inplace=True)
df_trans_mf.reset_index(drop="index", inplace=True)

In [29]:
def df_years_maker(df, new_col_name:str, df_with_iso=None):
    df_new = pd.DataFrame()
    df_tmp = pd.DataFrame()
    df_yearless = df.drop("years", axis=1)
    for col in df_yearless.columns:
        df_tmp = df_yearless[[col]]
        df_tmp["years"] = df["years"]
        df_tmp["country"] = col
        iso = df_with_iso.loc[df_with_iso["country"]==col, "iso3"]
        iso_string = str(iso.values[0])
        df_tmp["iso"] = iso_string
        df_tmp.rename(columns={col:new_col_name}, inplace=True)
        df_new = pd.concat([df_new, df_tmp])
        df_new.reset_index(drop="index", inplace=True)
        df_new
    
    return df_new

In [30]:
df_years_mf = df_years_maker(df_trans_mf, "mf", df)
df_years_mf

C:\Users\pigop\AppData\Local\Temp\ipykernel_12800\4294848072.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\pigop\AppData\Local\Temp\ipykernel_12800\4294848072.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\pigop\AppData\Local\Temp\ipykernel_12800\4294848072.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

,mf,years,country,iso
0,2.33,1990,Afghanistan,AFG
1,2.28,1991,Afghanistan,AFG
2,2.35,1992,Afghanistan,AFG
3,2.1,1993,Afghanistan,AFG
4,1.83,1994,Afghanistan,AFG
...,...,...,...,...
6235,2.26,2017,Zimbabwe,ZWE
6236,2.05,2018,Zimbabwe,ZWE
6237,1.21,2019,Zimbabwe,ZWE
6238,1.21,2020,Zimbabwe,ZWE


In [31]:
df_years_mf.mf = df_years_mf["mf"].astype(float)

In [32]:
fig = px.choropleth(df_years_mf, locations="iso",
                    color="mf", # lifeExp is a column of gapminder
                    hover_name="country", # column to add to hover information
                    color_continuous_scale=[(0, "white"), 
                        (0.33,"rgb(210,105,30)"), 
                        (0.66, "rgb(125,86,17)"), 
                        (1, "black")], 
                    animation_frame="years", 
                    range_color=[0,110])

fig.update_traces(marker_line_color='rgb(75, 75, 75)', marker_line_width=0.2)
fig.update_xaxes(tickangle=0)


fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    margin={"r":0, "t":0, "l":0, "b":100}, height=480,
)

fig['layout']['sliders'][0]['pad']=dict(l=10, t=10, r=-65)
fig['layout']['updatemenus'][0]['pad']=dict(l=0,t=13)
fig.show()
fig.write_html("../html/chart/map_box_mf.html")